In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.manage_precipitation_data import create_tp_daily_summary, create_tp_monthly_aggregate, create_list_lat_long

In [10]:
from pathlib import Path


def create_daily_rainfall_dataframe(file_path = './output', location = 'london', debug = False):
    # retrieve all files for location
    folder_path = Path('./output')
    string_in_filename = location + '_daily'
    location_files = [f for f in folder_path.glob('*.parquet') if string_in_filename in f.name]
    
    # open each files and create a list of dataframe
    dfs = []
    for file in location_files:
        df = pd.read_parquet(file)
        dfs.append(df)

    # concat list of dataframes
    df_all = pd.concat(dfs, axis = 0).sort_values('time_dt').reset_index(drop=True)
    if debug:
        n_points = len(create_list_lat_long(df_all))
        n_days = ((df_all['time_dt'].max() - df_all['time_dt'].min()).days+1)
        print(f"number of expected rows = {n_points*n_days}, number of returned rows = {df_all.shape[0]}")
    return(df_all)

def transform_data_datetime(df):
    df = df.copy()
    df['year'] = df['time_dt'].dt.year
    df['month int'] = df['time_dt'].dt.month
    df['month'] = df['time_dt'].dt.strftime('%b')
    df['month'] = pd.Categorical(df['month'], categories=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                                          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], ordered=True)

    df['day'] = df['time_dt'].dt.day


    return df

def thresholding_and_rounding(series, threshold = 0.1):
    s1 = series.copy()
    s1[s1 < threshold] = 0
    return(np.round(s1,1))

def create_monthly_aggregate(location = 'london', remove_date_list = ['2009-12-31'], debug = False):
    '''
    Create function signature    
    '''

    # first, create daily total precipitation for all points in the grid
    # in the area defined by location. This data is contained in pickle files 
    # from a specific folder. The function concat all yearly files created in
    # previous steps
    df_data = create_daily_rainfall_dataframe(location=location, debug=debug)
    if debug:
        print(df_data[df_data['time'] == df_data['time'].min()])
    # removing data contained in remove_date_list
    df_data = df_data[~df_data['time'].isin(remove_date_list)].reset_index(drop=True).copy()

    # creating time_dt column to process the mean
    df_data = transform_data_datetime(df_data)

    # Averaging measurements for all points in the grid
    df_data_mean = (df_data
                        .groupby('time_dt')['tp_mm']
                        .mean()
                        ).reset_index()

    # creating year, month, day colums
    df_data_mean = transform_data_datetime(df_data_mean)
    # thresholding and rounding tp_mm values
    df_data_mean['tp_mm'] = thresholding_and_rounding(df_data_mean['tp_mm'])
    # creating year-month aggregate - to decide if saving the result as a parquet file for further analysis on time/monthly series
    # At this stage, we are just using the monthly sum
    df_data_year_month = (df_data_mean
                            .groupby(['year', 'month int'])
                            .agg(
                                month = ('month', 'first'),
                                tp_monthly_sum = ('tp_mm', 'sum'),
                                tp_daily_median = ('tp_mm', 'median'),
                                tp_daily_avg = ('tp_mm', 'mean'),
                                tp_daily_std = ('tp_mm', 'std'),
                                tp_daily_min = ('tp_mm', 'min'),
                                tp_daily_max = ('tp_mm', 'max')
                                ).reset_index()
                                )
    
    # creating the 'year_month' as string variable - this is just if we want to save the resulting dataframe for further analysis
    df_data_year_month['year_month'] = df_data_year_month['year'].astype(str) + '-' + df_data_year_month['month int'].astype(str).str.zfill(2)
    
    filename_str = location + '_year_month_from_' + df_data_year_month['year_month'].min() + '_to_' + df_data_year_month['year_month'].max()
    if debug:
        print(filename_str)
    
    # finally, creating monthly aggregate

    data_monthly_tp = (df_data_year_month
                         .groupby('month', observed=True)
                         .agg(
                             month_sum = ('tp_monthly_sum', 'sum'),
                             month_median = ('tp_monthly_sum', 'median'),
                             month_avg = ('tp_monthly_sum', 'mean'),
                             month_std = ('tp_monthly_sum', 'std'),
                             month_min = ('tp_monthly_sum', 'min'),
                             month_max = ('tp_monthly_sum', 'max')
                             )
                             .reset_index()
                             )
    
    data_monthly_tp[['month_median', 'month_avg','month_std']] = np.round(data_monthly_tp[['month_median', 'month_avg','month_std']],1)    
    
    return(data_monthly_tp)

import matplotlib.pyplot as plt
import numpy as np

# Example data
def plot_monthly_aggregate_static(df_plot, feature_label = 'Mean', bar_width = 0.35, location = 'London'):
    label_dict = {'Mean':'month_avg', 'Median':'month_median', 'Sum':'month_sum', 'Avg' :'month_avg', 'Average':'month_avg'}
    feature = label_dict[feature_label]
    months = df_plot['month'].values
    # agg_precip = df_plot[feature].values
    min_precip = df_plot['month_min'].values
    max_precip = df_plot['month_max'].values

    # Convert months to x-axis positions
    x = np.arange(len(months))

    # Plot
    plt.figure(figsize=(10, 4))
    # plt.plot(x, agg_precip, label='Mean Precipitation', color='blue', marker='o')
    print(feature)
    ax = sns.barplot(data = df_plot, 
                x = 'month', 
                y = feature,
                width=0.5)

    plt.plot(x, max_precip, label='Max Precipitation', color='red', marker='o')
    plt.fill_between(x, min_precip, max_precip, color='lightblue', alpha=0.4, label='Min-Max Range')
    plt.plot(x, min_precip, label='Min Precipitation', color='green', marker='d')

    # Add annotations on top of each bar
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', label_type='edge', padding=3)

    # Move legend outside the plot
    ax.legend(
        title='Aggregate Values',
        bbox_to_anchor=(1.05, 1),  # x=1.05 moves it to the right
        loc='upper left',          # anchor point of the legend
        borderaxespad=0.
    )

    plt.xticks(x, months)
    plt.ylabel("Precipitation (mm)")
    plt.title(f"Monthly {feature_label} Rainfall with Min-Max Range in {location}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [12]:
london_monthly_aggregate = create_monthly_aggregate(remove_date_list = ['1999-12-31'],debug=True)
apulia_monthly_aggregate = create_monthly_aggregate('puglia',remove_date_list = ['1999-12-31'], debug=True)

number of expected rows = 111720, number of returned rows = 111720
          time  latitude     longitude        tp  tp_mm    time_dt
0   1999-12-31      51.6 -2.000000e-01  0.000115    0.1 1999-12-31
1   1999-12-31      51.4  1.000000e-01  0.000085    0.0 1999-12-31
2   1999-12-31      51.4  2.775558e-17  0.000079    0.0 1999-12-31
3   1999-12-31      51.4 -1.000000e-01  0.000090    0.0 1999-12-31
4   1999-12-31      51.5  1.000000e-01  0.000101    0.1 1999-12-31
5   1999-12-31      51.5  2.775558e-17  0.000094    0.0 1999-12-31
6   1999-12-31      51.4 -2.000000e-01  0.000101    0.1 1999-12-31
7   1999-12-31      51.5 -2.000000e-01  0.000108    0.1 1999-12-31
8   1999-12-31      51.6  1.000000e-01  0.000115    0.1 1999-12-31
9   1999-12-31      51.6  2.775558e-17  0.000106    0.1 1999-12-31
10  1999-12-31      51.6 -1.000000e-01  0.000110    0.1 1999-12-31
11  1999-12-31      51.5 -1.000000e-01  0.000101    0.1 1999-12-31
london_year_month_from_2000-01_to_2025-06
number of expected r

In [ ]:
create_monthly_aggregate(location = 'london', remove_date_list = ['1999-12-31'], debug = False)

,month,month_sum,month_median,month_avg,month_std,month_min,month_max
0,Jan,1546.9,57.6,59.5,25.2,17.9,114.4
1,Feb,1407.4,48.8,54.1,27.4,9.7,116.1
2,Mar,1304.3,45.9,50.2,28.3,12.1,114.9
3,Apr,1289.9,43.6,49.6,29.9,5.2,119.2
4,May,1594.8,54.6,61.3,27.4,6.3,110.7


In [14]:
london_monthly_aggregate.tail()

,month,month_sum,month_median,month_avg,month_std,month_min,month_max
7,Aug,1681.8,65.0,67.3,28.1,17.0,125.4
8,Sep,1214.1,47.8,48.6,27.0,15.4,132.4
9,Oct,1730.9,62.4,69.2,33.9,22.3,143.3
10,Nov,1699.2,66.3,68.0,30.1,13.3,119.4
11,Dec,1525.6,57.6,61.0,26.8,10.1,109.7


## Min and Max years analysis

We now want to create a column that, for each max and month, indicatest in which year it happened.

In [18]:
london_daily = create_daily_rainfall_dataframe(location='london', debug=False)
apulia_daily = create_daily_rainfall_dataframe(location='puglia', debug=False)

In [20]:
london_daily = transform_data_datetime(london_daily)

# Averaging measurements for all points in the grid
london_daily_mean = (london_daily
                    .groupby('time_dt')['tp_mm']
                    .mean()
                    ).reset_index()

# creating year, month, day colums
london_daily_mean = transform_data_datetime(london_daily_mean)
# thresholding and rounding tp_mm values
london_daily_mean['tp_mm'] = thresholding_and_rounding(london_daily_mean['tp_mm'])

In [21]:
london_daily_mean.head()

,time_dt,tp_mm,year,month int,month,day
0,1999-12-31,0.0,1999,12,Dec,31
1,2000-01-01,0.2,2000,1,Jan,1
2,2000-01-02,0.2,2000,1,Jan,2
3,2000-01-03,8.4,2000,1,Jan,3
4,2000-01-04,0.5,2000,1,Jan,4


In [22]:
london_year_month = (london_daily_mean
                        .groupby(['year', 'month int'])
                        .agg(
                            month = ('month', 'first'),
                            tp_monthly_sum = ('tp_mm', 'sum'),
                            tp_daily_median = ('tp_mm', 'median'),
                            tp_daily_avg = ('tp_mm', 'mean'),
                            tp_daily_std = ('tp_mm', 'std'),
                            tp_daily_min = ('tp_mm', 'min'),
                            tp_daily_max = ('tp_mm', 'max')
                            ).reset_index()
                            )


In [24]:
london_monthly_tp = (london_year_month
                         .groupby('month', observed=True)
                         .agg(
                             month_sum = ('tp_monthly_sum', 'sum'),
                             month_median = ('tp_monthly_sum', 'median'),
                             month_avg = ('tp_monthly_sum', 'mean'),
                             month_std = ('tp_monthly_sum', 'std'),
                             month_min = ('tp_monthly_sum', 'min'),
                             month_max = ('tp_monthly_sum', 'max')
                             )
                             .reset_index()
                             )

In [25]:
london_monthly_tp

,month,month_sum,month_median,month_avg,month_std,month_min,month_max
0,Jan,1546.9,57.60,59.496154,25.247043,17.9,114.4
1,Feb,1407.4,48.75,54.130769,27.423476,9.7,116.1
2,Mar,1304.3,45.90,50.165385,28.313007,12.1,114.9
3,Apr,1289.9,43.65,49.611538,29.907609,5.2,119.2
4,May,1594.8,54.65,61.338462,27.445788,6.3,110.7
5,Jun,1587.3,49.15,61.050000,36.503109,8.6,136.8
6,Jul,1693.9,70.20,67.756000,24.990366,16.3,116.5
7,Aug,1681.8,65.00,67.272000,28.138860,17.0,125.4
8,Sep,1214.1,47.80,48.564000,26.979713,15.4,132.4
9,Oct,1730.9,62.40,69.236000,33.935673,22.3,143.3


In [30]:
london_year_month[['year','month', 'tp_monthly_sum']].merge(london_monthly_tp['month_min'], how='inner', left_on='tp_monthly_sum', right_on='month_min').sort_values('month')

,year,month,tp_monthly_sum,month_min
1,2000,Jan,17.9,17.9
12,2023,Feb,9.7,9.7
5,2011,Mar,12.1,12.1
4,2007,Apr,6.3,6.3
6,2011,Apr,5.2,5.2
9,2020,May,6.3,6.3
8,2018,Jun,8.6,8.6
11,2022,Jul,16.3,16.3
2,2003,Aug,17.0,17.0
3,2003,Sep,15.4,15.4


In [ ]:
london_monthly_aggregate = create_monthly_aggregate(debug=True)
apulia_monthly_aggregate = create_monthly_aggregate('puglia', debug=True)

In [13]:
london_monthly_aggregate.to_parquet('../streamlit/data/london_monthly_aggregate.parquet')
apulia_monthly_aggregate.to_parquet('../streamlit/data/apulia_monthly_aggregate.parquet')

In [15]:
london_monthly_aggregate.head()

,month,month_sum,month_median,month_avg,month_std,month_min,month_max
0,Jan,1546.9,57.6,59.5,25.2,17.9,114.4
1,Feb,1407.4,48.8,54.1,27.4,9.7,116.1
2,Mar,1304.3,45.9,50.2,28.3,12.1,114.9
3,Apr,1289.9,43.6,49.6,29.9,5.2,119.2
4,May,1594.8,54.6,61.3,27.4,6.3,110.7


In [17]:
london_monthly_aggregate[london_monthly_aggregate['month_avg'] == london_monthly_aggregate['month_avg'].max()]

,month,month_sum,month_median,month_avg,month_std,month_min,month_max
9,Oct,1730.9,62.4,69.2,33.9,22.3,143.3


In [ ]:
apulia_monthly_aggregate

In [ ]:
plot_monthly_aggregate_static(apulia_monthly_aggregate, 'Mean')

In [ ]:
plot_monthly_aggregate_static(london_monthly_aggregate, 'Mean')

In [ ]:
import plotly.graph_objects as go


def plot_monthly_aggregate_dynamic(df_plot, 
                                   feature_label = 'Mean', 
                                   bar_width = 0.35, 
                                   location = 'London',
                                   fig_width = 800,
                                   fig_height = 400):
    # Create figure
    fig = go.Figure()

    label_dict = {'Mean':'month_avg', 'Median':'month_median', 'Sum':'month_sum', 'Avg' :'month_avg', 'Average':'month_avg'}
    feature = label_dict[feature_label]

    months = df_plot['month'].values
    mean_precip = df_plot[feature].values
    min_precip = df_plot['month_min'].values
    max_precip = df_plot['month_max'].values


    # Add min-max range as a filled area
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
            'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    fig.add_trace(go.Scatter(
        x=months + months[::-1],  # x: Jan to Dec and back from Dec to Jan
        y=list(max_precip) + list(min_precip[::-1]),  # Upper bound + lower bound reversed
        fill='toself',
        fillcolor='rgba(135, 206, 250, 0.4)',  # Light blue with transparency
        line=dict(color='rgba(255,255,255,0)'),  # No line
        hoverinfo="skip",
        showlegend=True,
        name='Min-Max Range'
    ))

    # Add mean line
    # fig.add_trace(go.Scatter(
    #     x=months,
    #     y=mean_precip,
    #     mode='lines+markers',
    #     name='Mean Precipitation',
    #     line=dict(color='blue'),
    #     marker=dict(size=6)
    # ))

    fig.add_trace(go.Bar(
        x = months,
        y = mean_precip,
        name='Mean Precipitation',
        marker=dict(
            color='blue',
            line=dict(width=0),
            opacity=0.9
        ),
        width=bar_width
    ))

    # Layout
    fig.update_layout(
        title = f"Monthly {feature_label} Rainfall with Min-Max Range in {location}",
        xaxis_title='Month',
        yaxis_title='Precipitation (mm)',
        template='plotly_white',
        hovermode="x unified",
        barmode='overlay',
        barcornerradius=8,
        width=fig_width,
        height=fig_height,
    )

    return(fig)

In [ ]:
plot_monthly_aggregate_dynamic(london_monthly_aggregate, 'Mean', bar_width=0.65, fig_width=1000, fig_height=600)

In [ ]:
def plot_monthly_aggregate_dynamic_V2(df_plot, 
                                   feature_label='Mean', 
                                   bar_width=0.35, 
                                   location='London',
                                   fig_width=800,
                                   fig_height=400):
    fig = go.Figure()

    # Label mapping
    label_dict = {'Mean': 'month_avg', 'Median': 'month_median', 'Sum': 'month_sum', 'Avg': 'month_avg', 'Average': 'month_avg'}
    feature = label_dict[feature_label]

    # Months and data
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    mean_precip = df_plot[feature].values
    min_precip = df_plot['month_min'].values
    max_precip = df_plot['month_max'].values

    # --- Min-Max filled area ---
    fig.add_trace(go.Scatter(
        x=months + months[::-1],  # Jan→Dec + Dec→Jan
        y=list(max_precip) + list(min_precip[::-1]),
        fill='toself',
        fillcolor='rgba(135, 206, 250, 0.3)',  # Transparent blue
        line=dict(color='rgba(255,255,255,0)'),  # No border line
        hoverinfo='skip',
        showlegend=True,
        name='Min-Max Range'
    ))

    # --- Bar for Mean/Median/Sum ---
    fig.add_trace(go.Bar(
        x=months,
        y=mean_precip,
        name=f'{feature_label} Precipitation',
        marker=dict(
            color='blue',
            opacity=0.9
        ),
        width=bar_width
    ))

    # --- Scatter for Min ---
    fig.add_trace(go.Scatter(
        x=months,
        y=min_precip,
        mode='markers',
        name='Min',
        marker=dict(
            symbol='circle',
            color='dodgerblue',
            size=8
        )
    ))

    # --- Scatter for Max ---
    fig.add_trace(go.Scatter(
        x=months,
        y=max_precip,
        mode='markers',
        name='Max',
        marker=dict(
            symbol='diamond',
            color='darkblue',
            size=8
        )
    ))

    # --- Layout with grid ---
    fig.update_layout(
        title=f"Monthly {feature_label} Rainfall with Min-Max Range in {location}",
        xaxis_title='Month',
        yaxis_title='Precipitation (mm)',
        template='plotly_white',
        hovermode="x unified",
        barmode='overlay',
        barcornerradius=8,
        width=fig_width,
        height=fig_height,
        xaxis=dict(
            showgrid=True,
            gridcolor='lightgray'
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='lightgray'
        ),
    )

    return fig

In [ ]:
plot_monthly_aggregate_dynamic_V2(london_monthly_aggregate, 'Mean', bar_width=0.65, fig_width=1000, fig_height=600)

In [ ]:
import plotly.graph_objects as go

def plot_monthly_aggregate_dynamic_v3(df_plot, 
                                   feature_label='Mean', 
                                   bar_width=0.35, 
                                   location='London',
                                   fig_width=800,
                                   fig_height=400,
                                   dark_mode=False):
    fig = go.Figure()

    # Label mapping
    label_dict = {
        'Mean': 'month_avg', 'Median': 'month_median',
        'Sum': 'month_sum', 'Avg': 'month_avg', 'Average': 'month_avg'
    }
    feature = label_dict[feature_label]

    # Consistent month order
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    mean_precip = df_plot[feature].values
    min_precip = df_plot['month_min'].values
    max_precip = df_plot['month_max'].values

    # --- Colorblind-safe palette ---
    bar_color = '#0072B2'        # royal blue
    min_color = '#D55E00'        # vermillion
    max_color = '#009E73'        # bluish green
    fill_color = 'rgba(100, 100, 100, 0.2)' if not dark_mode else 'rgba(200, 200, 200, 0.2)'
    grid_color = 'gray' if dark_mode else 'lightgray'
    text_color = 'white' if dark_mode else 'black'

    # --- Filled area ---
    fig.add_trace(go.Scatter(
        x=months + months[::-1],
        y=list(max_precip) + list(min_precip[::-1]),
        fill='toself',
        fillcolor=fill_color,
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=True,
        name='Min-Max Range'
    ))

    # --- Bar plot ---
    fig.add_trace(go.Bar(
        x=months,
        y=mean_precip,
        name=f'{feature_label}',
        marker=dict(
            color=bar_color,
            opacity=0.9
        ),
        width=bar_width,
        hovertemplate=f'{feature_label}: %{{y:.1f}} mm<extra></extra>'
    ))

    # --- Min line+markers ---
    fig.add_trace(go.Scatter(
        x=months,
        y=min_precip,
        mode='lines+markers',
        name='Min',
        line=dict(color=min_color, width=2, dash='dot'),
        marker=dict(symbol='circle', size=7, color=min_color),
        hovertemplate=f'Min: %{{y:.1f}} mm<extra></extra>'
    ))

    # --- Max line+markers ---
    fig.add_trace(go.Scatter(
        x=months,
        y=max_precip,
        mode='lines+markers',
        name='Max',
        line=dict(color=max_color, width=2),
        marker=dict(symbol='diamond', size=8, color=max_color),
        hovertemplate=f'Max: %{{y:.1f}} mm<extra></extra>'
    ))

    # --- Layout ---
    fig.update_layout(
        title=f"Monthly {feature_label} Rainfall with Min-Max Range in {location}",
        xaxis_title='Month',
        yaxis_title='Precipitation (mm)',
        template='plotly_dark' if dark_mode else 'plotly_white',
        hovermode="x unified",
        barmode='overlay',
        barcornerradius=8,
        width=fig_width,
        height=fig_height,
        font=dict(color=text_color),
        xaxis=dict(
            showgrid=True,
            gridcolor=grid_color,
            zeroline=False
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor=grid_color,
            zeroline=False
        ),
        legend=dict(
            bgcolor='rgba(0,0,0,0)' if dark_mode else 'rgba(255,255,255,0)',
            borderwidth=0
        )
    )

    return fig


In [ ]:
plot_monthly_aggregate_dynamic_v3(london_monthly_aggregate, 'Average', bar_width=0.65, fig_width=1000, fig_height=600, dark_mode=False)

In [ ]:
import plotly.graph_objects as go

def plot_monthly_aggregate_comparison(df1, df2,
                                      label1='Dataset 1',
                                      label2='Dataset 2',
                                      feature_label='Mean',
                                      bar_width=0.35,
                                      location='Comparison',
                                      fig_width=900,
                                      fig_height=500,
                                      dark_mode=False):
    fig = go.Figure()

    # Label mapping
    label_dict = {'Mean': 'month_avg', 'Median': 'month_median',
                  'Sum': 'month_sum', 'Avg': 'month_avg', 'Average': 'month_avg'}
    feature = label_dict[feature_label]

    # Shared months axis
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # --- Colorblind-safe palette ---
    # Dataset 1
    bar_color1 = '#0072B2'     # royal blue
    min_color1 = '#D55E00'     # vermillion
    max_color1 = '#009E73'     # bluish green

    # Dataset 2
    bar_color2 = '#E69F00'     # orange
    min_color2 = '#56B4E9'     # sky blue
    max_color2 = '#CC79A7'     # reddish pink

    fill_color = 'rgba(120, 120, 120, 0.15)' if not dark_mode else 'rgba(200, 200, 200, 0.15)'
    grid_color = 'gray' if dark_mode else 'lightgray'
    text_color = 'white' if dark_mode else 'black'

    # --- Dataset 1 values ---
    mean1 = df1[feature].values
    min1 = df1['month_min'].values
    max1 = df1['month_max'].values

    # --- Dataset 2 values ---
    mean2 = df2[feature].values
    min2 = df2['month_min'].values
    max2 = df2['month_max'].values

    # === Dataset 1 ===
    fig.add_trace(go.Scatter(
        x=months + months[::-1],
        y=list(max1) + list(min1[::-1]),
        fill='toself',
        fillcolor=fill_color,
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=True,
        name=f'{label1} Min-Max Range'
    ))

    fig.add_trace(go.Bar(
        x=months,
        y=mean1,
        name=f'{label1} {feature_label}',
        marker=dict(color=bar_color1, opacity=0.85),
        width=bar_width,
        hovertemplate=f'{label1} {feature_label}: %{{y:.1f}}<extra></extra>'
    ))

    fig.add_trace(go.Scatter(
        x=months,
        y=min1,
        mode='lines+markers',
        name=f'{label1} Min',
        line=dict(color=min_color1, width=2, dash='dot'),
        marker=dict(symbol='circle', size=7, color=min_color1),
        hovertemplate=f'{label1} Min: %{{y:.1f}}<extra></extra>'
    ))

    fig.add_trace(go.Scatter(
        x=months,
        y=max1,
        mode='lines+markers',
        name=f'{label1} Max',
        line=dict(color=max_color1, width=2),
        marker=dict(symbol='diamond', size=8, color=max_color1),
        hovertemplate=f'{label1} Max: %{{y:.1f}}<extra></extra>'
    ))

    # === Dataset 2 ===
    fig.add_trace(go.Scatter(
        x=months + months[::-1],
        y=list(max2) + list(min2[::-1]),
        fill='toself',
        fillcolor=fill_color,
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=True,
        name=f'{label2} Min-Max Range'
    ))

    fig.add_trace(go.Bar(
        x=months,
        y=mean2,
        name=f'{label2} {feature_label}',
        marker=dict(color=bar_color2, opacity=0.85),
        width=bar_width,
        hovertemplate=f'{label2} {feature_label}: %{{y:.1f}}<extra></extra>'
    ))

    fig.add_trace(go.Scatter(
        x=months,
        y=min2,
        mode='lines+markers',
        name=f'{label2} Min',
        line=dict(color=min_color2, width=2, dash='dot'),
        marker=dict(symbol='circle', size=7, color=min_color2),
        hovertemplate=f'{label2} Min: %{{y:.1f}}<extra></extra>'
    ))

    fig.add_trace(go.Scatter(
        x=months,
        y=max2,
        mode='lines+markers',
        name=f'{label2} Max',
        line=dict(color=max_color2, width=2),
        marker=dict(symbol='diamond', size=8, color=max_color2),
        hovertemplate=f'{label2} Max: %{{y:.1f}}<extra></extra>'
    ))

    # === Layout ===
    fig.update_layout(
        title=f"Monthly {feature_label} Rainfall Comparison in {location}",
        xaxis_title='Month',
        yaxis_title='Precipitation (mm)',
        template='plotly_dark' if dark_mode else 'plotly_white',
        hovermode='x unified',
        barmode='overlay',  # alternatives: 'group', 'relative'
        barcornerradius=6,
        width=fig_width,
        height=fig_height,
        font=dict(color=text_color),
        xaxis=dict(
            showgrid=True,
            gridcolor=grid_color
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor=grid_color
        ),
        legend=dict(
            bgcolor='rgba(0,0,0,0)' if dark_mode else 'rgba(255,255,255,0)',
            borderwidth=0
        )
    )

    return fig

In [ ]:
plot_monthly_aggregate_comparison(london_monthly_aggregate, apulia_monthly_aggregate,
                                      label1='London',
                                      label2='Apulia',
                                      feature_label='Mean',
                                      bar_width=0.35,
                                      location='Comparison',
                                      fig_width=900,
                                      fig_height=500,
                                      dark_mode=False)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_monthly_aggregate_stacked(df1, df2,
                                   label1='Dataset 1',
                                   label2='Dataset 2',
                                   feature_label='Mean',
                                   bar_width=0.35,
                                   location='Comparison',
                                   fig_width=900,
                                   fig_height=800,
                                   dark_mode=False):
    # Setup subplots: 2 rows, shared x-axis
    fig = make_subplots(rows=2, cols=1,
                        shared_xaxes=True,
                        vertical_spacing=0.1,
                        subplot_titles=(f"{label1} Monthly {feature_label}", f"{label2} Monthly {feature_label}"))

    label_dict = {'Mean': 'month_avg', 'Median': 'month_median',
                  'Sum': 'month_sum', 'Avg': 'month_avg', 'Average': 'month_avg'}
    feature = label_dict[feature_label]

    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # --- Colorblind-safe palette ---
    bar_color1 = '#0072B2'; min_color1 = '#D55E00'; max_color1 = '#009E73'
    bar_color2 = '#E69F00'; min_color2 = '#56B4E9'; max_color2 = '#CC79A7'
    fill_color = 'rgba(120, 120, 120, 0.15)' if not dark_mode else 'rgba(200, 200, 200, 0.15)'
    grid_color = 'gray' if dark_mode else 'lightgray'
    text_color = 'white' if dark_mode else 'black'

    # === Plot Dataset 1 ===
    mean1 = df1[feature].values
    min1 = df1['month_min'].values
    max1 = df1['month_max'].values

    fig.add_trace(go.Scatter(
        x=months + months[::-1],
        y=list(max1) + list(min1[::-1]),
        fill='toself',
        fillcolor=fill_color,
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=False,
        name=f'{label1} Min-Max Range'
    ), row=1, col=1)

    fig.add_trace(go.Bar(
        x=months,
        y=mean1,
        name=f'{label1} {feature_label}',
        marker=dict(color=bar_color1, opacity=0.85),
        width=bar_width,
        hovertemplate=f'{label1} {feature_label}: %{{y:.1f}}<extra></extra>'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=months,
        y=min1,
        mode='lines+markers',
        name=f'{label1} Min',
        line=dict(color=min_color1, width=2, dash='dot'),
        marker=dict(symbol='circle', size=7, color=min_color1),
        hovertemplate=f'{label1} Min: %{{y:.1f}}<extra></extra>'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=months,
        y=max1,
        mode='lines+markers',
        name=f'{label1} Max',
        line=dict(color=max_color1, width=2),
        marker=dict(symbol='diamond', size=8, color=max_color1),
        hovertemplate=f'{label1} Max: %{{y:.1f}}<extra></extra>'
    ), row=1, col=1)

    # === Plot Dataset 2 ===
    mean2 = df2[feature].values
    min2 = df2['month_min'].values
    max2 = df2['month_max'].values

    fig.add_trace(go.Scatter(
        x=months + months[::-1],
        y=list(max2) + list(min2[::-1]),
        fill='toself',
        fillcolor=fill_color,
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=False,
        name=f'{label2} Min-Max Range'
    ), row=2, col=1)

    fig.add_trace(go.Bar(
        x=months,
        y=mean2,
        name=f'{label2} {feature_label}',
        marker=dict(color=bar_color2, opacity=0.85),
        width=bar_width,
        hovertemplate=f'{label2} {feature_label}: %{{y:.1f}}<extra></extra>'
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=months,
        y=min2,
        mode='lines+markers',
        name=f'{label2} Min',
        line=dict(color=min_color2, width=2, dash='dot'),
        marker=dict(symbol='circle', size=7, color=min_color2),
        hovertemplate=f'{label2} Min: %{{y:.1f}}<extra></extra>'
    ), row=2, col=1)

    fig.add_trace(go.Scatter(
        x=months,
        y=max2,
        mode='lines+markers',
        name=f'{label2} Max',
        line=dict(color=max_color2, width=2),
        marker=dict(symbol='diamond', size=8, color=max_color2),
        hovertemplate=f'{label2} Max: %{{y:.1f}}<extra></extra>'
    ), row=2, col=1)

    # === Layout ===
    fig.update_layout(
        title=f"{feature_label} Monthly Rainfall in {location}",
        xaxis_title='Month',
        yaxis_title='Precipitation (mm)',
        template='plotly_dark' if dark_mode else 'plotly_white',
        hovermode='x unified',
        width=fig_width,
        height=fig_height,
        font=dict(color=text_color),
        xaxis=dict(showgrid=True, gridcolor=grid_color),
        xaxis2=dict(showgrid=True, gridcolor=grid_color),
        yaxis=dict(showgrid=True, gridcolor=grid_color),
        yaxis2=dict(showgrid=True, gridcolor=grid_color),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.15,
            xanchor="center",
            x=0.5,
            bgcolor='rgba(0,0,0,0)' if dark_mode else 'rgba(255,255,255,0)',
            borderwidth=0
        )
    )

    return fig

In [ ]:
plot_monthly_aggregate_stacked(london_monthly_aggregate, apulia_monthly_aggregate,
                                   label1='London',
                                   label2='Apulia',
                                   feature_label='Mean',
                                   bar_width=0.35,
                                   location='Comparison',
                                   fig_width=900,
                                   fig_height=800,
                                   dark_mode=True)

In [ ]:
import plotly.graph_objects as go

def plot_monthly_aggregate_overlaid(df1, df2,
                                    label1='Dataset 1',
                                    label2='Dataset 2',
                                    feature_label='Mean',
                                    bar_width=0.35,
                                    location='Comparison',
                                    fig_width=1000,
                                    fig_height=500,
                                    dark_mode=False):
    fig = go.Figure()

    label_dict = {'Mean': 'month_avg', 'Median': 'month_median',
                  'Sum': 'month_sum', 'Avg': 'month_avg', 'Average': 'month_avg'}
    feature = label_dict[feature_label]

    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

    # Colorblind-safe palette
    bar_color1 = '#0072B2'; min_color1 = '#D55E00'; max_color1 = '#009E73'
    bar_color2 = '#E69F00'; min_color2 = '#56B4E9'; max_color2 = '#CC79A7'
    fill_color1 = 'rgba(100, 100, 100, 0.1)'
    fill_color2 = 'rgba(150, 150, 150, 0.1)'
    grid_color = 'gray' if dark_mode else 'lightgray'
    text_color = 'white' if dark_mode else 'black'

    # Dataset 1 values
    mean1 = df1[feature].values
    min1 = df1['month_min'].values
    max1 = df1['month_max'].values

    # Dataset 2 values
    mean2 = df2[feature].values
    min2 = df2['month_min'].values
    max2 = df2['month_max'].values

    # --- Fill Areas (Optional) ---
    fig.add_trace(go.Scatter(
        x=months + months[::-1],
        y=list(max1) + list(min1[::-1]),
        fill='toself',
        fillcolor=fill_color1,
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=True,
        name=f'{label1} Min-Max Range'
    ))

    fig.add_trace(go.Scatter(
        x=months + months[::-1],
        y=list(max2) + list(min2[::-1]),
        fill='toself',
        fillcolor=fill_color2,
        line=dict(color='rgba(0,0,0,0)'),
        hoverinfo='skip',
        showlegend=True,
        name=f'{label2} Min-Max Range'
    ))

    # --- Bars for Dataset 1 ---
    fig.add_trace(go.Bar(
        x=months,
        y=mean1,
        name=f'{label1} {feature_label}',
        marker=dict(color=bar_color1),
        width=bar_width,
        offsetgroup=0,
        hovertemplate=f'{label1} {feature_label}: %{{y:.1f}}<extra></extra>'
    ))

    # --- Bars for Dataset 2 ---
    fig.add_trace(go.Bar(
        x=months,
        y=mean2,
        name=f'{label2} {feature_label}',
        marker=dict(color=bar_color2),
        width=bar_width,
        offsetgroup=1,
        hovertemplate=f'{label2} {feature_label}: %{{y:.1f}}<extra></extra>'
    ))

    # --- Min/Max lines for Dataset 1 ---
    fig.add_trace(go.Scatter(
        x=months,
        y=min1,
        mode='lines+markers',
        name=f'{label1} Min',
        line=dict(color=min_color1, width=2, dash='dot'),
        marker=dict(symbol='circle', size=7, color=min_color1),
        hovertemplate=f'{label1} Min: %{{y:.1f}}<extra></extra>'
    ))

    fig.add_trace(go.Scatter(
        x=months,
        y=max1,
        mode='lines+markers',
        name=f'{label1} Max',
        line=dict(color=max_color1, width=2),
        marker=dict(symbol='diamond', size=8, color=max_color1),
        hovertemplate=f'{label1} Max: %{{y:.1f}}<extra></extra>'
    ))

    # --- Min/Max lines for Dataset 2 ---
    fig.add_trace(go.Scatter(
        x=months,
        y=min2,
        mode='lines+markers',
        name=f'{label2} Min',
        line=dict(color=min_color2, width=2, dash='dot'),
        marker=dict(symbol='circle', size=7, color=min_color2),
        hovertemplate=f'{label2} Min: %{{y:.1f}}<extra></extra>'
    ))

    fig.add_trace(go.Scatter(
        x=months,
        y=max2,
        mode='lines+markers',
        name=f'{label2} Max',
        line=dict(color=max_color2, width=2),
        marker=dict(symbol='diamond', size=8, color=max_color2),
        hovertemplate=f'{label2} Max: %{{y:.1f}}<extra></extra>'
    ))

    # --- Layout ---
    fig.update_layout(
        title=f"{feature_label} Monthly Rainfall Comparison in {location}",
        xaxis_title='Month',
        yaxis_title='Precipitation (mm)',
        template='plotly_dark' if dark_mode else 'plotly_white',
        hovermode='x unified',
        barmode='group',  # group instead of overlay
        width=fig_width,
        height=fig_height,
        font=dict(color=text_color),
        xaxis=dict(
            showgrid=True,
            gridcolor=grid_color,
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor=grid_color,
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=-0.3,
            xanchor="center",
            x=0.5,
            bgcolor='rgba(0,0,0,0)' if dark_mode else 'rgba(255,255,255,0)'
        )
    )

    return fig

In [ ]:
plot_monthly_aggregate_overlaid(london_monthly_aggregate, apulia_monthly_aggregate,
                                   label1='London',
                                   label2='Apulia',
                                   feature_label='Mean',
                                   bar_width=0.35,
                                   location='Comparison',
                                   fig_width=900,
                                   fig_height=800,
                                   dark_mode=True)

In [ ]:
london_monthly_aggregate[['month_median', 'month_avg','month_std']] = np.round(london_monthly_aggregate[['month_median', 'month_avg','month_std']],1)
london_monthly_aggregate.head()

In [ ]:
london_monthly_aggregate

In [ ]:
apulia_monthly_aggregate

In [ ]:
df_all_puglia = create_daily_rainfall_dataframe(location='puglia', debug=True)
df_all_london = create_daily_rainfall_dataframe(location='london', debug=True)

# still have one day from 2009 - removing it
df_all_london = df_all_london[df_all_london['time'] != '2009-12-31'].reset_index(drop=True).copy()
df_all_puglia = df_all_puglia[df_all_puglia['time'] != '2009-12-31'].reset_index(drop=True).copy()

In [ ]:
df_all_london.head()

# Rainfall Monthly and Seasonal Analysis

In this notebook we will perform monthly and seasonal total precipitation analysis and exploring:
1. how total precipitation is characterised during the different months of the year and how they compare between London area and Apulia
2. with respect to season, which are the ones where it rains more and which less
3. is there any change in the last 50 years - that's for London area only.

As for point 1. and 2., we will compare how it does change between London and Apulia area, while for point 3., we will refer to London are only for the moment

In [ ]:
df_all_london.head()

In [ ]:
apulia_data_all_points = transform_data_datetime(df_all_puglia)
london_data_all_points = transform_data_datetime(df_all_london)

### Monthly sum, mean, median, mean and max

As we want to analyse the monthly precipitation data, we want to measure what was the montly total precipitaion sum, mean, median, min and max in the 2 different areas.

In [ ]:
df_london_daily_mean = (df_all_london
                        .groupby('time_dt')['tp_mm']
                        .mean()
                        ).reset_index()

df_london_daily_mean.head()

In [ ]:
df_london_daily_mean = transform_data_datetime(df_london_daily_mean)

In [ ]:
df_london_daily_mean.head(3)

In [ ]:
def thresholding_and_rounding(series, threshold = 0.1):
    s1 = series.copy()
    s1[s1 < threshold] = 0
    return(np.round(s1,1))

In [ ]:
df_london_daily_mean['tp_mm'] = thresholding_and_rounding(df_london_daily_mean['tp_mm'])

In [ ]:
df_london_year_month = (df_london_daily_mean
 .groupby(['year', 'month int'])
 .agg(
     month = ('month', 'first'),
     tp_monthly_sum = ('tp_mm', 'sum'),
     tp_daily_median = ('tp_mm', 'median'),
     tp_daily_avg = ('tp_mm', 'mean'),
     tp_daily_std = ('tp_mm', 'std'),
     tp_daily_min = ('tp_mm', 'min'),
     tp_daily_max = ('tp_mm', 'max')
 ).reset_index()
)

df_london_year_month['year_month'] = df_london_year_month['year'].astype(str) + '-' + df_london_year_month['month int'].astype(str).str.zfill(2)
df_london_year_month.head()

In [ ]:
sns.set_theme(rc={'figure.figsize':(14.,4.)})

df_plot = df_london_year_month[['year_month', 'tp_monthly_sum']].copy()
# df_plot['year_month'] = pd.to_datetime(df_plot['year_month'], format = '%Y-%m')
# df_plot = df_plot.sort_values('year_month')

sns.barplot(
    data = df_plot,
    x = 'year_month',
    y = 'tp_monthly_sum'
);

## Show only every 6th label
xticks = df_plot['year_month'].unique()
plt.xticks(
    ticks=range(0, len(xticks), 6),
    labels=xticks[::6],
    rotation=90
)

plt.xlabel('Year-Month')
plt.ylabel('tp_mm')
plt.tight_layout()
plt.show()

In [ ]:
df_london_year_month.head()

In [ ]:
london_monthly_tp = (df_london_year_month
 .groupby('month', observed=True)
 .agg(
     month_sum = ('tp_monthly_sum', 'sum'),
     month_median = ('tp_monthly_sum', 'mean'),
     month_avg = ('tp_monthly_sum', 'mean'),
     month_std = ('tp_monthly_sum', 'std'),
     month_min = ('tp_monthly_sum', 'min'),
     month_max = ('tp_monthly_sum', 'max')
     )
     .reset_index()
 )

In [ ]:
london_monthly_tp

In [ ]:
sns.barplot(
    data = london_monthly_tp,
    x= 'month',
    y = 'month_median'
)

sns.scatterplot(
    data = london_monthly_tp,
    x= 'month',
    y = 'month_max'
)
sns.lineplot(
    data = london_monthly_tp,
    x= 'month',
    y = 'month_max',
    color = 'green'
)

sns.scatterplot(
    data = london_monthly_tp,
    x= 'month',
    y = 'month_min'
)
sns.lineplot(
    data = london_monthly_tp,
    x= 'month',
    y = 'month_min',
    color = 'orange'
)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
months = london_monthly_tp['month'].values
mean_precip = london_monthly_tp['month_avg'].values
min_precip = london_monthly_tp['month_min'].values
max_precip = london_monthly_tp['month_max'].values

# Convert months to x-axis positions
x = np.arange(len(months))

# Plot
plt.figure(figsize=(10, 4))
plt.plot(x, mean_precip, label='Mean Precipitation', color='blue', marker='o')
plt.fill_between(x, min_precip, max_precip, color='lightblue', alpha=0.4, label='Min-Max Range')

plt.xticks(x, months)
plt.ylabel("Precipitation (mm)")
plt.title("Monthly Mean Rainfall with Min-Max Range")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
max_precip

In [ ]:
import plotly.graph_objects as go
# Create figure
fig = go.Figure()

# Add min-max range as a filled area
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig.add_trace(go.Scatter(
    x=months + months[::-1],  # x: Jan to Dec and back from Dec to Jan
    y=list(max_precip) + list(min_precip[::-1]),  # Upper bound + lower bound reversed
    fill='toself',
    fillcolor='rgba(135, 206, 250, 0.4)',  # Light blue with transparency
    line=dict(color='rgba(255,255,255,0)'),  # No line
    hoverinfo="skip",
    showlegend=True,
    name='Min-Max Range'
))

# Add mean line
# fig.add_trace(go.Scatter(
#     x=months,
#     y=mean_precip,
#     mode='lines+markers',
#     name='Mean Precipitation',
#     line=dict(color='blue'),
#     marker=dict(size=6)
# ))

fig.add_trace(go.Bar(
    x = months,
    y = mean_precip,
    name='Mean Precipitation',
    marker=dict(
        color='blue',
        line=dict(width=0),
        opacity=0.9
    ),
    width=0.3
))

# Layout
fig.update_layout(
    title='Monthly Rainfall: Mean with Min-Max Range',
    xaxis_title='Month',
    yaxis_title='Precipitation (mm)',
    template='plotly_white',
    hovermode="x unified",
    barmode='overlay',
    barcornerradius=8
)

fig.show()

In [ ]:
sns.boxplot(
    data = df_london_year_month,
    x = 'month',
    y = 'tp_monthly_sum'
)

## Apulia Monthly Analysis

In [ ]:
df_apulia_daily_mean = (df_all_puglia
                        .groupby('time_dt')['tp_mm']
                        .mean()
                        ).reset_index()


df_apulia_daily_mean = transform_data_datetime(df_apulia_daily_mean)
df_apulia_daily_mean['tp_mm'] = thresholding_and_rounding(df_apulia_daily_mean['tp_mm'])

df_apulia_year_month = (df_apulia_daily_mean
 .groupby(['year', 'month int'])
 .agg(
     month = ('month', 'first'),
     tp_monthly_sum = ('tp_mm', 'sum'),
     tp_daily_median = ('tp_mm', 'median'),
     tp_daily_avg = ('tp_mm', 'mean'),
     tp_daily_std = ('tp_mm', 'std'),
     tp_daily_min = ('tp_mm', 'min'),
     tp_daily_max = ('tp_mm', 'max')
 ).reset_index()
)

df_apulia_year_month['year_month'] = df_apulia_year_month['year'].astype(str) + '-' + df_apulia_year_month['month int'].astype(str).str.zfill(2)

apulia_monthly_tp = (df_apulia_year_month
 .groupby('month', observed=True)
 .agg(
     month_sum = ('tp_monthly_sum', 'sum'),
     month_median = ('tp_monthly_sum', 'mean'),
     month_avg = ('tp_monthly_sum', 'mean'),
     month_std = ('tp_monthly_sum', 'std'),
     month_min = ('tp_monthly_sum', 'min'),
     month_max = ('tp_monthly_sum', 'max')
     )
     .reset_index()
 )

apulia_monthly_tp


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
months = apulia_monthly_tp['month'].values
mean_precip = apulia_monthly_tp['month_avg'].values
min_precip = apulia_monthly_tp['month_min'].values
max_precip = apulia_monthly_tp['month_max'].values

# Convert months to x-axis positions
x = np.arange(len(months))

# Plot
plt.figure(figsize=(10, 4))
plt.plot(x, mean_precip, label='Mean Precipitation', color='blue', marker='o')
plt.fill_between(x, min_precip, max_precip, color='lightblue', alpha=0.4, label='Min-Max Range')

plt.xticks(x, months)
plt.ylabel("Precipitation (mm)")
plt.title("Monthly Mean Rainfall with Min-Max Range")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

## `create_monthly_aggregate` - function creating Monthly Aggregate from daily rainfall data

The function takes as an input the location of the area we want to create the Monthly Aggregate dataframe. 

The output dataframe will have the following format: 

| Month | sum    |  tp_mm_median | tp_mm_avg | tp_mm_std | tp_mm_min | tp_mm_max |
|------:|:-------|:--------------|:----------|:----------|:----------|:----------|
| Jan   | 2404.5 |   26.6        | 18.1      |  4.3      |   1.3     |  50.3     |
| ...   |   ...  |   ...         | ...       |  ...      |   ...     |  ...      |

In [ ]:
def create_monthly_aggregate(location = 'london', remove_date_list = ['2009-12-31'], debug = False):
    '''
    Create function signature    
    '''

    # first, create daily total precipitation for all points in the grid
    # in the area defined by location. This data is contained in pickle files 
    # from a specific folder. The function concat all yearly files created in
    # previous steps
    df_data = create_daily_rainfall_dataframe(location=location, debug=debug)
    # removing data contained in remove_date_list
    df_data = df_data[~df_data['time'].isin(remove_date_list)].reset_index(drop=True).copy()

    # creating time_dt column to process the mean
    df_data = transform_data_datetime(df_data)

    # Averaging measurements for all points in the grid
    df_data_mean = (df_data
                        .groupby('time_dt')['tp_mm']
                        .mean()
                        ).reset_index()

    # creating year, month, day colums
    df_data_mean = transform_data_datetime(df_data_mean)
    # thresholding and rounding tp_mm values
    df_data_mean['tp_mm'] = thresholding_and_rounding(df_data_mean['tp_mm'])
    # creating year-month aggregate - to decide if saving the result as a parquet file for further analysis on time/monthly series
    # At this stage, we are just using the monthly sum
    df_data_year_month = (df_data_mean
                            .groupby(['year', 'month int'])
                            .agg(
                                month = ('month', 'first'),
                                tp_monthly_sum = ('tp_mm', 'sum'),
                                tp_daily_median = ('tp_mm', 'median'),
                                tp_daily_avg = ('tp_mm', 'mean'),
                                tp_daily_std = ('tp_mm', 'std'),
                                tp_daily_min = ('tp_mm', 'min'),
                                tp_daily_max = ('tp_mm', 'max')
                                ).reset_index()
                                )
    
    # creating the 'year_month' as string variable - this is just if we want to save the resulting dataframe for further analysis
    df_data_year_month['year_month'] = df_data_year_month['year'].astype(str) + '-' + df_data_year_month['month int'].astype(str).str.zfill(2)
    
    filename_str = location + '_year_month_from_' + df_data_year_month['year_month'].min() + '_to_' + df_data_year_month['year_month'].max()
    if debug:
        print(filename_str)
    
    # finally, creating monthly aggregate

    data_monthly_tp = (df_data_year_month
                         .groupby('month', observed=True)
                         .agg(
                             month_sum = ('tp_monthly_sum', 'sum'),
                             month_median = ('tp_monthly_sum', 'median'),
                             month_avg = ('tp_monthly_sum', 'mean'),
                             month_std = ('tp_monthly_sum', 'std'),
                             month_min = ('tp_monthly_sum', 'min'),
                             month_max = ('tp_monthly_sum', 'max')
                             )
                             .reset_index()
                             )

    
    return(data_monthly_tp)

In [ ]:
test_apulia = create_monthly_aggregate(location='puglia')
test_apulia.head()

In [ ]:
test_london = create_monthly_aggregate(location='london')
test_london.head()

## Visualizing monthly aggregate

We have two different functions to visualize monthly sum/median/avg with min and max values using matplotlib and plotly for static and dinamyc visualization.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Example data
def plot_monthly_aggregate_static(df_plot, feature_label = 'Mean', bar_width = 0.35, location = 'London'):
    label_dict = {'Mean':'month_avg', 'Median':'month_median', 'Sum':'month_sum', 'Avg' :'month_avg', 'Average':'month_avg'}
    feature = label_dict[feature_label]
    months = df_plot['month'].values
    # agg_precip = df_plot[feature].values
    min_precip = df_plot['month_min'].values
    max_precip = df_plot['month_max'].values

    # Convert months to x-axis positions
    x = np.arange(len(months))

    # Plot
    plt.figure(figsize=(10, 4))
    # plt.plot(x, agg_precip, label='Mean Precipitation', color='blue', marker='o')
    print(feature)
    ax = sns.barplot(data = df_plot, 
                x = 'month', 
                y = feature,
                width=0.5)

    plt.plot(x, max_precip, label='Max Precipitation', color='red', marker='o')
    plt.fill_between(x, min_precip, max_precip, color='lightblue', alpha=0.4, label='Min-Max Range')
    plt.plot(x, min_precip, label='Min Precipitation', color='green', marker='d')

    # Add annotations on top of each bar
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', label_type='edge', padding=3)

    # Move legend outside the plot
    ax.legend(
        title='Aggregate Values',
        bbox_to_anchor=(1.05, 1),  # x=1.05 moves it to the right
        loc='upper left',          # anchor point of the legend
        borderaxespad=0.
    )

    plt.xticks(x, months)
    plt.ylabel("Precipitation (mm)")
    plt.title(f"Monthly {feature_label} Rainfall with Min-Max Range in {location}")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_monthly_aggregate_static(test_london, feature_label='Sum')

In [ ]:
k = {'mean':'month_avg', 'median':'month_median', 'sum':'month_sum'}
k['mean']

## Notes: ***How often it rains in London or Apulia region?***

I have a question: I want to analyse the number of dry days, i.e. the number of days withour raining. As I have measurements for 12 different points in the grid of London, we want to check if by calculating the mean of precipitation considering the 12 points is different if we consider each point separately. What I mean, if in one particular point only of the grid I measure some rain, how it effects the number of dry days with compare the mean of all 12 points. This is particularly interesting if we are considering a larger area like a region in a country like Apulia. In a larger region, I can end up by considering much less dry day if I consider the mean for all points then finding a way to define dry days for the region. To illustrate better this point, let's assume that in the north part of Apulia region I have a rainy day in the first 182 days and no rain for the rest of the year, and the opposite happens in the south part. If I take the precipitation mean, I will conlcude that there are no dry days in the Apulia region even if in some parts there have been more than 180 days without rain. So, considering the daily mean for each point, does not give you the right idea of how often it rains in Apulia.

Let's start with London area and see what data report.

First I will consider how many dry days I have for each singular point. And then I will calculate the number of dry days after computing the mean for each day.


In [ ]:
london_data_all_points.head()

In [ ]:
london_data_all_points['rainy_day'] = np.where(london_data_all_points['tp_mm'] > 0, 1, 0)

In [ ]:
london_rainy_days_points = london_data_all_points.groupby('time')['rainy_day'].sum().reset_index()

In [ ]:
london_rainy_days_points.head()

In [ ]:
(london_rainy_days_points[london_rainy_days_points['rainy_day'] == 0].shape[0] + london_rainy_days_points[london_rainy_days_points['rainy_day'] == 12].shape[0])/london_rainy_days_points.shape[0]

In [ ]:
sns.histplot(
    data = london_rainy_days_points,
    x = 'rainy_day',
    bins = 12
);

The plot shows that there are few instances where all 12 points in the grid report rain or not. The 90% of the data show that we have measured rain or not in all 12 different points of the grid.

In [ ]:
apulia_data_all_points['rainy_day'] = np.where(apulia_data_all_points['tp_mm'] > 0, 1, 0)
apulia_data_all_points.head()

In [ ]:
apulia_rainy_days_points = apulia_data_all_points.groupby('time')['rainy_day'].sum().reset_index()
apulia_rainy_days_points.head()

In [ ]:
sns.set_theme(rc={'figure.figsize':(14.,4.)})

sns.histplot(
    data = apulia_rainy_days_points,
    x = 'rainy_day',
    bins = 177
);

In [ ]:
sns.set_theme(rc={'figure.figsize':(14.,4.)})

sns.histplot(
    data = apulia_rainy_days_points[apulia_rainy_days_points['rainy_day'] <= 25],
    x = 'rainy_day',
    bins = 25
);

In [ ]:
london_daily_data = pd.read_pickle('../streamlit/data/london_tp_daily_data.pkl')
london_daily_data.shape

In [ ]:
london_daily_data.head()

In [ ]:
london_daily_data.dtypes

In [ ]:
(london_daily_data['date'].max() - london_daily_data['date'].min()).days

In [ ]:
(london_daily_data
 .groupby('month')
 .agg(
     tp_mm_daily_month_avg = ('tp_mm_daily_sum', 'mean'),
     tp_mm_daily_month_min = ('tp_mm_daily_sum', 'min'),
     tp_mm_daily_month_max = ('tp_mm_daily_sum', 'max'),
 )
 ).reset_index()

In [ ]:
(london_daily_data
 .groupby(['year', 'month'])
 .agg(
     tp_mm_month_sum = ('tp_mm_daily_sum', 'sum'),
     tp_mm_daily_mean = ('tp_mm_daily_sum', 'mean'),
     tp_mm_daily_min = ('tp_mm_daily_sum', 'min'),
     tp_mm_daily_max = ('tp_mm_daily_sum', 'max'),
 ).reset_index()
)

In [ ]:
year = 1999
month = 'Aug'

london_daily_data[(london_daily_data['year'] == year) &
                  (london_daily_data['month'] == month)][['day', 'tp_mm_daily_sum']]

In [ ]:
year = 2015

sns.set_theme(rc={'figure.figsize':(14.,4.)})

sns.scatterplot(
    data = london_daily_data[london_daily_data['year'] == year],
    x = 'date',
    y = 'tp_mm_daily_sum'
);

In [ ]:
max_tp_per_year = (london_daily_data
 .groupby('year')
 .agg(
     max_tp = ('tp_mm_daily_sum', 'max')
     )
     .rename(columns = {'max_tp':'tp_mm_daily_sum'})
 ).reset_index()


max_tp_per_year

In [ ]:
max_tp_per_year.merge(london_daily_data[['year', 'tp_mm_daily_sum','month']], how = 'left')

In [ ]:
(max_tp_per_year.merge(london_daily_data[['year', 'tp_mm_daily_sum','month']], how = 'left')
 .groupby('month')['year']
 .count()
).reset_index().sort_values('year', ascending = False)